# comfyui 工程及插件安装


In [ ]:
# install comfyui 
%cd /kaggle/working
!git clone https://github.com/comfyanonymous/ComfyUI.git
%cd ComfyUI
!pip install -r requirements.txt

comfyui = '/kaggle/working/ComfyUI'
model_path = '/kaggle/working/ComfyUI/models'
temp = '/kaggle/temp'
!mkdir -p $temp
!mv $model_path $temp
temp_models = '/kaggle/temp/models'
get_ipython().system(f'ln -s {temp_models} {model_path}')


# install custom nodes
custom_nodes = '/kaggle/working/ComfyUI/custom_nodes'
%cd $custom_nodes
!git clone https://github.com/ltdrdata/ComfyUI-Manager.git
%cd ComfyUI-Manager
!pip install -r requirements.txt

%cd $custom_nodes
!git clone https://github.com/ZHO-ZHO-ZHO/ComfyUI-Flowty-TripoSR-ZHO.git
%cd ComfyUI-Flowty-TripoSR-ZHO
#  install torchmcubes 
!pip install git+https://github.com/tatsy/torchmcubes.git@3aef8afa5f21b113afc4f4ea148baee850cbd472
!pip install -r requirements.txt

%cd $custom_nodes
!git clone https://github.com/AlekPet/ComfyUI_Custom_Nodes_AlekPet.git

# install openssh
!mamba install openssh -y

!echo 'env is ready'

In [ ]:
# install RMBG & models
%cd $custom_nodes
!git clone https://github.com/ZHO-ZHO-ZHO/ComfyUI-BRIA_AI-RMBG.git
!cd ComfyUI-BRIA_AI-RMBG
%rm -rf RMBG-1.4
!git clone https://huggingface.co/briaai/RMBG-1.4

# 模型下载

从hungingface下载模型,记得将以下的hf_*** token换成你自己的

In [ ]:
%cd $temp_models
# 从hungingface下载模型, 下载方式参考文档https://padeoe.com/huggingface-large-models-downloader/
!mkdir -p checkpoints
# download Playground v2.5
repo = ' https://huggingface.co/playgroundai/playground-v2.5-1024px-aesthetic/resolve/main%s?download=true'
model_dict = {
    '/playground-v2.5-1024px-aesthetic.fp16.safetensors': './checkpoints/playground-v25.safetensors',
}

for k,v in model_dict.items():
        cmd = f'wget --header "Authorization: Bearer hf_***" {repo % k} -O {v}'
        print(cmd)
        get_ipython().system(cmd)

In [ ]:
%cd $temp_models
# 从hungingface下载模型, 下载方式参考文档https://padeoe.com/huggingface-large-models-downloader/
!mkdir -p controlnet
# download controlnet
repo = ' https://huggingface.co/diffusers/controlnet-canny-sdxl-1.0/resolve/main%s?download=true'
model_dict = {
    '/diffusion_pytorch_model.fp16.safetensors': './controlnet/controlnet-canny-sdxl-1.0.safetensors',
}

for k,v in model_dict.items():
        cmd = f'wget --header "Authorization: Bearer hf_***" {repo % k} -O {v}'
        print(cmd)
        get_ipython().system(cmd)

# 启动服务

找到输出日志中带有.pinggy.link 后缀的链接，通过浏览器打开

In [ ]:
# Starting the Web UI with pinggy

from multiprocessing import Process
import sys
import time

!touch log.txt
open('log.txt', 'w').close()

def run_app():
    cmd = f"python /kaggle/working/ComfyUI/main.py & ssh -o StrictHostKeyChecking=no -p 80 -R0:localhost:8188 a.pinggy.io > log.txt"
    get_ipython().system(cmd)
    
def print_url():
    print("waiting for output")
    time.sleep(2)
    sys.stdout.flush()
    
    found = False
    with open('log.txt', 'r') as file:
        end_word = '.pinggy.link'
        for line in file:
            start_index = line.find("http:")
            if start_index != -1:
                end_index = line.find(end_word, start_index)
                if end_index != -1:
                    print("😁 😁 😁")
                    print("URL: " + line[start_index:end_index + len(end_word)])
                    print("😁 😁 😁")
                    found = True
    if not found:
        print_url()
    else:
        with open('log.txt', 'r') as file:
            for line in file:
                print(line)
    
p_app = Process(target=run_app)
p_url = Process(target=print_url)
p_app.start()
p_url.start()
p_app.join()
p_url.join()

# 工作流下载

工作流json文件，从[ComfyUI-Workflows-ZHO](https://github.com/ZHO-ZHO-ZHO/ComfyUI-Workflows-ZHO) 仓库中获取Sketch to 3D【Zho】的json文件